In [1]:
import pandas as pd
import numpy as np
import re
import os
import shutil
from Tools import tools

In [2]:
# Initialize lists to store the data
event_names = []
event_types = []
predictions = []

# Regular expression pattern to match the data
pattern = r'bn(\d+)\s+(\w+)\s+\[([\d.e\-\s]+)\]'

# Read the data and extract information
data = '''
bn110222659 UNCERT [1.0000000e+00 2.0652371e-14 7.7315329e-15 2.4463487e-14]
bn240323372 UNCERT [9.9237108e-01 4.9943681e-04 1.4001724e-03 5.7293461e-03]
bn090407780 UNCERT [1.0000000e+00 2.8554006e-10 1.6803917e-09 9.8042419e-09]
bn131016664 UNCERT [1.0000000e+00 1.6849875e-13 6.8798214e-15 4.0638125e-13]
bn110102432 UNCERT [9.9977177e-01 2.7596350e-09 6.6443114e-08 2.2813222e-04]
bn240628601 UNCERT [9.2004341e-01 7.3024586e-02 3.5975783e-04 6.5722787e-03]
bn221018317 UNCERT [0.9061992  0.07968655 0.00123819 0.012876  ]
bn120716030 UNRELOC [9.8878342e-01 6.8209099e-04 3.5924322e-04 1.0175230e-02]
bn130628931 BELOWHZ [9.9967790e-01 2.1796502e-04 1.3945416e-05 9.0214024e-05]
bn211014070 UNCERT [9.9816376e-01 2.9325741e-04 4.7011170e-04 1.0729086e-03]
bn091022752 UNCERT [9.9997711e-01 2.0795505e-06 1.3184894e-05 7.5430803e-06]
bn230904491 UNCERT [1.0000000e+00 6.3237708e-09 1.4762901e-11 1.8833221e-08]
bn230913474 UNCERT [9.9998796e-01 3.1769578e-06 6.9301683e-07 8.1166236e-06]
bn220115787 UNCERT [9.9957567e-01 1.2574295e-04 3.2771663e-06 2.9525047e-04]
bn240524337 UNCERT [9.9135196e-01 5.6551206e-03 6.8081974e-04 2.3121680e-03]
bn240618962 UNCERT [9.7017306e-01 6.9507267e-05 2.8497201e-05 2.9729011e-02]
bn110130345 UNCERT [9.9985898e-01 1.9889412e-05 3.0515006e-05 9.0659050e-05]
bn091208623 UNCERT [9.9988914e-01 5.9771792e-05 1.9970488e-05 3.0997839e-05]
bn160929461 UNCERT [9.9982494e-01 8.3949271e-06 3.0047436e-06 1.6374209e-04]
bn230420570 UNCERT [9.0741265e-01 8.7124027e-02 2.3584966e-05 5.4397550e-03]
bn210807895 UNRELOC [9.999876e-01 2.941325e-06 7.230316e-06 2.176839e-06]
bn120605953 BELOWHZ [0.96303016 0.00655822 0.00365379 0.02675778]
bn230213324 UNCERT [9.7295940e-01 1.4310431e-02 2.8775423e-04 1.2442441e-02]
bn130205970 UNCERT [1.0000000e+00 1.3929480e-12 4.7324471e-13 3.0177768e-12]
bn170416538 UNRELOC [1.2407866e-27 1.0000000e+00 1.4984266e-29 1.2489068e-27]
bn240521839 UNCERT [5.8349408e-04 9.9410915e-01 1.9564922e-03 3.3509005e-03]
bn140704063 UNCERT [4.2431033e-08 9.9999797e-01 1.5661113e-06 5.2884269e-07]
bn211213088 UNCERT [0.0130097  0.97451824 0.0045687  0.00790345]
bn240513668 UNCERT [7.2027033e-04 9.2499679e-01 4.0147297e-02 3.4135591e-02]
bn190620772 UNRELOC [0.00480869 0.9882902  0.00168366 0.00521748]
bn150925345 UNCERT [1.5061334e-04 9.9971193e-01 7.7026561e-05 6.0399594e-05]
bn150424918 UNCERT [1.8542120e-05 9.9995625e-01 7.5921430e-06 1.7640161e-05]
bn190217091 UNRELOC [0.0262404  0.92678297 0.02075041 0.02622627]
bn170911787 UNCERT [3.5332126e-04 9.9942482e-01 1.0906932e-04 1.1288099e-04]
bn200518800 UNRELOC [2.1837843e-15 1.4993230e-12 8.2523964e-16 1.0000000e+00]
bn211029782 UNCERT [0.00148137 0.99227494 0.0028551  0.00338863]
bn240502886 UNCERT [1.06493610e-13 1.08709855e-13 1.47354868e-07 9.99999881e-01]
bn221029511 UNCERT [0.02686138 0.9414587  0.01039472 0.02128523]
bn131120031 UNCERT [5.8492813e-05 9.9955302e-01 6.1242448e-05 3.2714498e-04]
bn160616436 UNCERT [7.0778160e-06 9.9981040e-01 1.3394680e-04 4.8633545e-05]
bn160210750 UNCERT [2.0623658e-23 1.0000000e+00 2.7464713e-26 3.9514283e-22]
bn240409849 UNCERT [1.8444599e-06 9.9980289e-01 1.1473945e-05 1.8380578e-04]
bn210403738 UNCERT [2.2795335e-04 9.9474782e-01 5.9289648e-04 4.4312994e-03]
bn190901828 UNCERT [4.5674622e-10 8.5511917e-11 9.4752322e-09 1.0000000e+00]
bn190712514 UNCERT [1.4693098e-08 9.9999309e-01 3.6905767e-08 6.9495400e-06]
bn240508924 UNCERT [3.0326984e-05 8.1667783e-16 9.6643347e-19 9.9996972e-01]
bn100607804 UNCERT [3.5397104e-24 1.0000000e+00 6.6963389e-24 9.5960568e-23]
bn131122417 UNCERT [3.5202125e-04 1.3538128e-04 1.7334893e-04 9.9933928e-01]
bn220305986 UNCERT [2.8243148e-05 9.2771109e-03 7.5368490e-03 9.8315781e-01]
bn161121783 UNCERT [1.5952411e-06 9.9998116e-01 5.8154745e-07 1.6694459e-05]
bn140117640 UNCERT [8.5928396e-04 9.7329021e-01 2.2954756e-02 2.8957804e-03]
bn200930384 UNCERT [2.7875957e-04 9.9728549e-01 1.6685817e-03 7.6719630e-04]
bn211108091 UNCERT [2.0843252e-04 9.9766785e-01 9.2389068e-04 1.1997462e-03]
bn200118510 UNCERT [3.8105017e-04 9.9825960e-01 6.1197008e-04 7.4738520e-04]
bn210929724 UNCERT [0.0036325  0.9904703  0.00139512 0.00450212]
bn240421624 UNCERT [1.6915685e-02 9.8052788e-01 1.4483661e-04 2.4116805e-03]
bn150623507 UNCERT [1.1298207e-05 9.9904364e-01 1.1273759e-04 8.3233789e-04]
bn220118515 UNCERT [0.01704086 0.9700712  0.00840602 0.00448194]
bn220106928 UNCERT [0.02074755 0.9643487  0.00529161 0.00961204]
bn140520124 UNCERT [1.4302949e-09 9.9999988e-01 1.8382357e-08 8.0852992e-08]
bn210914975 UNCERT [4.3956636e-04 9.9179989e-01 2.5970377e-03 5.1635928e-03]
bn170423179 UNRELOC [0.0000000e+00 2.9835466e-38 1.6570139e-36 1.0000000e+00]
bn210425089 UNCERT [5.8268796e-04 9.9598545e-01 9.1937184e-04 2.5124224e-03]
bn170413037 UNCERT [8.930447e-08 9.999964e-01 9.005968e-08 3.330801e-06]
bn220511843 UNCERT [2.1908227e-03 9.8912734e-01 6.1580382e-04 8.0660796e-03]
bn181104784 UNCERT [0. 0. 0. 1.]
bn170319724 UNRELOC [1.0801157e-02 9.8603362e-01 2.4608008e-04 2.9191019e-03]
bn230302065 UNCERT [5.5529566e-05 9.9983168e-01 5.2707521e-05 6.0100039e-05]
bn121230875 UNCERT [5.2762033e-07 9.9992597e-01 5.3059034e-07 7.2921539e-05]
bn110519623 UNCERT [6.9595787e-26 1.0000000e+00 9.0180865e-26 2.6026119e-24]
bn190306960 UNCERT [8.2027889e-04 9.9668413e-01 1.1087408e-03 1.3867556e-03]
bn171003256 UNCERT [5.2493141e-07 9.9998081e-01 7.4569806e-07 1.7874478e-05]
bn171216812 UNCERT [1.4095506e-27 1.0000000e+00 6.7761829e-30 2.5874212e-28]
bn221013100 UNCERT [4.0587370e-06 9.9999595e-01 3.9403463e-12 3.1675239e-11]
bn220218845 UNCERT [0.00138195 0.98538476 0.00207988 0.01115341]
bn230623728 UNCERT [3.9131019e-06 9.9786538e-01 8.1243477e-04 1.3182666e-03]
bn150316434 UNCERT [5.8855032e-05 9.9986982e-01 3.3172862e-05 3.8196082e-05]
bn211209082 UNCERT [6.3514249e-10 3.8898138e-13 1.0000000e+00 1.1064533e-12]
bn130430050 UNCERT [6.7413286e-14 1.0000000e+00 1.8719504e-13 8.6101504e-13]
bn201027369 UNCERT [1.9154937e-03 9.9449182e-01 7.4330322e-04 2.8494662e-03]
bn160121343 UNCERT [0. 0. 0. 1.]
bn230805180 UNCERT [1.5581939e-11 2.2137691e-08 7.5951495e-11 1.0000000e+00]
bn230119980 UNCERT [6.0577207e-04 9.9278235e-01 4.2299307e-03 2.3819290e-03]
bn150318010 UNCERT [0.01304034 0.05447762 0.00988237 0.9225996 ]
bn091216359 BELOWHZ [6.1845756e-10 1.1976723e-11 1.1865463e-12 1.0000000e+00]
bn090129784 UNCERT [5.4097224e-08 2.4843868e-05 1.6020256e-09 9.9997520e-01]
bn140512120 UNRELOC [0.02755978 0.94511944 0.00670867 0.02061212]
bn150711543 UNCERT [5.5626046e-26 7.3197125e-28 5.8927788e-22 1.0000000e+00]
bn150530081 UNCERT [0.00246875 0.9949462  0.00101006 0.00157502]
bn110222255 UNCERT [7.3987212e-05 9.9973911e-01 3.5263296e-05 1.5159548e-04]
bn170912841 UNCERT [0.00350674 0.9914486  0.00214247 0.00290217]
bn121115044 BELOWHZ [6.6744322e-24 2.0958467e-34 4.4295199e-33 1.0000000e+00]
bn130828248 BELOWHZ [5.4079329e-04 2.5277914e-04 3.8464385e-04 9.9882179e-01]
bn210527628 UNRELOC [4.2002583e-08 1.3993989e-18 1.2625426e-15 1.0000000e+00]
bn170427592 UNRELOC [2.1151677e-03 9.9549758e-01 6.1218650e-04 1.7751181e-03]
bn150908768 UNCERT [1.5340116e-05 9.9984932e-01 7.3169822e-06 1.2798478e-04]
bn230926161 UNCERT [3.9675268e-03 9.9361104e-01 1.8374951e-04 2.2377055e-03]
bn161111083 UNCERT [1.6676255e-04 9.9880731e-01 1.9098553e-04 8.3492359e-04]
bn181215061 UNRELOC [1.1442662e-05 2.1098497e-04 2.4098145e-07 9.9977738e-01]
bn171019218 BELOWHZ [0. 0. 0. 1.]
bn230218466 UNCERT [0.00127422 0.00298391 0.9947174  0.00102441]
bn100227067 UNCERT [2.25831362e-04 1.00730234e-04 9.26851571e-05 9.99580801e-01]
bn131117611 UNCERT [4.5235009e-35 1.0000000e+00 3.3364274e-34 1.9009124e-30]
bn191207909 UNCERT [3.2227472e-08 9.9999976e-01 2.3850594e-08 2.3195484e-07]
bn161113521 UNCERT [1.0477781e-05 9.9991953e-01 6.1372061e-06 6.3867839e-05]
bn150717136 UNCERT [0.00131213 0.99243987 0.00220473 0.00404325]
bn170422374 UNRELOC [0. 0. 0. 1.]
bn161211422 UNRELOC [2.8929664e-04 3.4090113e-02 3.1597108e-05 9.6558893e-01]
bn190809554 UNCERT [6.6480477e-04 9.9582362e-01 2.0188664e-03 1.4926993e-03]
bn110124881 UNCERT [7.4838167e-06 9.9995244e-01 2.3774121e-06 3.7710695e-05]
bn151230686 UNCERT [2.9551671e-04 9.9383461e-01 5.3344238e-06 5.8646416e-03]
bn191025926 UNCERT [2.8603681e-35 1.0000000e+00 1.6158529e-36 7.6962940e-34]
bn220411732 UNCERT [5.6290213e-07 9.9999380e-01 5.0457811e-06 5.7717494e-07]
bn210114895 UNCERT [5.3750184e-05 9.9968719e-01 8.6203399e-05 1.7286986e-04]
bn191028919 UNCERT [5.63881913e-05 9.99674916e-01 1.64109879e-04 1.04593804e-04]
bn200731919 UNRELOC [7.5169147e-08 9.9999750e-01 3.9253500e-08 2.3310424e-06]
bn130828319 UNRELOC [3.9994204e-09 8.9968644e-08 3.1530444e-08 9.9999988e-01]
bn140504334 UNCERT [5.5887583e-10 1.0000000e+00 8.2083096e-10 2.2442647e-08]
bn150223935 UNRELOC [5.9120408e-03 4.8337647e-04 1.9356115e-02 9.7424853e-01]
bn240130149 UNCERT [1.5358216e-08 9.9999619e-01 1.1262848e-07 3.7439836e-06]
bn240505487 UNCERT [0.02014885 0.96372914 0.00557503 0.01054693]
bn240519588 UNCERT [4.6950345e-06 9.9996054e-01 2.9222219e-06 3.1829193e-05]
bn230528007 UNCERT [5.3804273e-05 9.9977845e-01 5.3299238e-05 1.1436102e-04]
bn210818917 UNCERT [6.1610226e-05 3.7353844e-09 9.9993837e-01 2.4847086e-08]
bn160831686 UNCERT [3.6949514e-06 9.9995983e-01 3.2404100e-06 3.3281056e-05]
bn230408607 UNCERT [1.7134136e-09 1.0000000e+00 1.9909676e-09 5.0315285e-09]
bn240311250 UNCERT [8.5327652e-04 9.9752384e-01 1.7008677e-04 1.4527462e-03]
bn171025203 UNCERT [1.274101e-03 9.967409e-01 1.357051e-03 6.280202e-04]
bn230330665 UNCERT [0.00267456 0.97136486 0.01982876 0.00613184]
bn190525500 UNCERT [4.0630907e-02 9.0240711e-01 5.0222274e-04 5.6459740e-02]
bn120702671 UNCERT [8.7211346e-03 9.8999435e-01 6.2398984e-05 1.2222378e-03]
bn230613572 UNCERT [2.0439882e-05 9.9991584e-01 7.2570901e-06 5.6469653e-05]
bn221214088 UNCERT [2.22747258e-04 9.96602774e-01 1.21442725e-04 3.05308448e-03]
bn151015141 UNCERT [5.1135306e-05 9.0648973e-01 3.1904962e-02 6.1554193e-02]
bn210508936 UNCERT [2.7656534e-03 9.9340844e-01 9.2248119e-05 3.7336075e-03]
bn180827314 UNCERT [9.5514132e-04 9.9722171e-01 8.0489856e-04 1.0182962e-03]
bn190114515 UNCERT [4.7068016e-04 9.9863178e-01 2.4973400e-04 6.4770610e-04]
bn220620347 UNCERT [2.1099623e-08 9.9999678e-01 2.7387275e-06 4.5297242e-07]
bn151207480 UNCERT [3.5323703e-06 9.9990726e-01 3.9993516e-05 4.9194135e-05]
bn100908160 UNCERT [0.00172404 0.99273294 0.00211061 0.00343247]
bn220818583 UNCERT [2.1193362e-07 9.9999642e-01 1.4814349e-06 1.8739401e-06]
bn240626761 UNCERT [2.1846990e-24 1.0000000e+00 3.7453294e-25 1.7217979e-23]
bn100306791 UNCERT [2.1567435e-03 9.9528474e-01 2.3789406e-04 2.3206489e-03]
bn170211197 UNCERT [2.4086809e-04 9.9784851e-01 3.5475834e-05 1.8751306e-03]
bn150916829 UNRELOC [4.8276235e-07 9.9999225e-01 1.8924736e-06 5.3596391e-06]
bn130325784 UNRELOC [1.1634160e-09 9.9999976e-01 2.7452860e-08 1.8065110e-07]
bn220626590 UNCERT [1.2851174e-31 1.0000000e+00 7.0325977e-33 1.3859739e-33]
bn220828664 BELOWHZ [0. 0. 0. 1.]
bn190624976 UNCERT [2.3148404e-04 9.9128598e-01 7.2704046e-03 1.2120923e-03]
bn151023076 UNCERT [6.6848695e-03 9.9044597e-01 8.7415590e-04 1.9950618e-03]
bn140513553 UNCERT [0.00617427 0.98415554 0.00241931 0.00725087]
bn140809988 UNCERT [0.01480444 0.964012   0.0118318  0.00935183]
bn200206815 UNCERT [7.3630479e-04 9.9768269e-01 8.7083556e-04 7.1014784e-04]
bn150426958 UNCERT [7.4813522e-07 2.9924006e-08 1.8868011e-08 9.9999928e-01]
bn181004148 UNCERT [3.693097e-04 9.993344e-01 1.278646e-04 1.684174e-04]
bn190620579 UNCERT [1.5981882e-03 9.9540722e-01 9.7092293e-04 2.0236718e-03]
bn130225458 UNCERT [1.2578680e-06 9.9994528e-01 7.1317834e-07 5.2735737e-05]
bn180402734 UNRELOC [5.8723117e-06 9.9615943e-01 1.6128826e-05 3.8185199e-03]
bn180223328 UNCERT [5.3924858e-04 9.5839137e-01 2.5061084e-04 4.0818803e-02]
bn211012795 UNCERT [6.3649903e-04 9.9347442e-01 1.2220899e-03 4.6669790e-03]
bn240308556 UNCERT [2.6451430e-06 9.9498808e-01 3.5414142e-03 1.4679348e-03]
bn221011605 UNCERT [1.0802031e-03 9.9875879e-01 2.4385745e-05 1.3664320e-04]
bn181213038 UNCERT [3.3396453e-02 9.6542990e-01 1.9721061e-04 9.7636896e-04]
bn220924543 UNCERT [4.5911435e-04 9.9855417e-01 1.8247943e-05 9.6846471e-04]
bn210713697 UNCERT [3.6008723e-04 1.6468310e-03 1.5581545e-04 9.9783725e-01]
bn150917046 UNCERT [4.2628814e-03 5.8520104e-09 9.1849410e-07 9.9573612e-01]
bn171026019 UNCERT [2.1222826e-04 1.6641909e-05 1.6762631e-05 9.9975437e-01]
bn220808197 UNCERT [4.9738344e-05 9.9840003e-01 5.2807946e-04 1.0221241e-03]
bn230913430 UNCERT [1.2521851e-06 9.9945050e-01 5.2118988e-04 2.7096061e-05]
bn221129787 UNCERT [3.1177379e-04 9.9900395e-01 1.8041967e-04 5.0385034e-04]
bn240429949 UNCERT [2.4549381e-04 9.9885046e-01 5.5326556e-04 3.5080052e-04]
bn200516818 UNRELOC [1.9994238e-03 9.8772484e-01 2.4413479e-04 1.0031596e-02]
bn240415875 UNCERT [0.00101053 0.99274904 0.00122492 0.0050156 ]
bn170603580 UNCERT [4.9597950e-04 9.9760568e-01 1.4790262e-03 4.1927607e-04]
bn210521213 UNRELOC [8.6905613e-18 2.5355446e-15 1.0000000e+00 2.8894191e-20]
bn221206544 UNRELOC [3.91678877e-12 1.00000000e+00 1.14179916e-10 3.70573772e-09]
bn240126368 UNCERT [3.0884374e-04 9.9924481e-01 1.1719200e-04 3.2913167e-04]
bn230909003 UNCERT [3.3235900e-02 9.6328139e-01 1.6568339e-04 3.3170511e-03]
bn180519511 UNCERT [1.4037140e-07 9.9999857e-01 1.6653691e-07 1.0959533e-06]
bn200304740 UNCERT [1.0033149e-05 9.9919146e-01 1.8268604e-04 6.1580498e-04]
bn160216355 UNCERT [2.2489512e-08 9.9999940e-01 1.8511113e-09 6.3142096e-07]
bn140925412 UNCERT [6.6324233e-08 9.9999893e-01 6.4638726e-08 9.9468377e-07]
bn240528727 UNCERT [5.1848799e-12 2.2017373e-05 2.3932560e-15 9.9997795e-01]
bn150914103 UNCERT [1.4587419e-31 1.3192935e-29 3.6016951e-26 1.0000000e+00]
bn140706303 UNCERT [1.7480816e-04 1.3810780e-09 2.1184483e-03 9.9770677e-01]
bn240319326 UNCERT [2.3123107e-04 9.9794418e-01 2.4136997e-04 1.5831656e-03]
bn151024950 UNCERT [2.4244364e-03 9.9583352e-01 4.1153358e-04 1.3305261e-03]
bn230717048 UNCERT [1.4993289e-07 9.9999952e-01 1.1830635e-08 3.9643768e-07]
bn240508902 UNCERT [0.000000e+00 9.777491e-37 0.000000e+00 1.000000e+00]
bn150907725 UNCERT [2.6122318e-05 9.9937975e-01 9.6593372e-05 4.9750874e-04]
bn210311226 UNCERT [1.3480700e-06 9.9998140e-01 6.8187169e-06 1.0322038e-05]
bn191226233 UNCERT [1.3909764e-04 9.9956185e-01 1.6729829e-05 2.8232127e-04]
bn210111515 UNCERT [8.2086108e-04 9.8161143e-01 9.3998229e-03 8.1678946e-03]
bn240119536 UNCERT [0.00226935 0.9555527  0.02688933 0.01528869]
bn170528284 UNRELOC [6.138803e-20 0.000000e+00 0.000000e+00 1.000000e+00]
bn231114681 UNCERT [5.8527377e-07 9.9999106e-01 1.2549982e-06 7.1126874e-06]
bn130301705 UNCERT [1.4451766e-07 9.9999857e-01 2.4695282e-10 1.2899897e-06]
bn210411821 UNCERT [6.2994608e-03 9.8942381e-01 6.7842973e-04 3.5984241e-03]
bn230302317 UNCERT [2.4740136e-04 9.9925166e-01 2.0755011e-04 2.9340631e-04]
bn200223717 UNCERT [0.00625879 0.98829365 0.00177683 0.00367072]
bn210914498 UNCERT [0.00778796 0.9723465  0.01525709 0.00460844]
bn240521936 UNCERT [5.5340433e-22 3.1631846e-18 2.6522213e-33 1.0000000e+00]
bn150124745 UNRELOC [8.8519453e-05 9.9961060e-01 1.0200048e-04 1.9888484e-04]
bn180904763 UNCERT [5.1907905e-06 9.9995553e-01 1.1934226e-05 2.7327642e-05]
bn231207255 UNCERT [7.8074620e-05 9.9969888e-01 8.3810737e-05 1.3927706e-04]
bn131205428 UNCERT [5.5633597e-03 9.8655570e-01 8.2442345e-04 7.0565483e-03]
bn171107017 UNRELOC [2.7857994e-25 1.9729228e-22 1.3644804e-19 1.0000000e+00]
bn170528148 BELOWHZ [9.2166593e-29 4.6848892e-37 1.0000000e+00 6.0080010e-35]
bn171130749 UNRELOC [2.6949700e-05 9.9879658e-01 9.7750526e-05 1.0787193e-03]
bn160317596 BELOWHZ [0. 0. 0. 1.]
bn100405414 UNCERT [6.258735e-35 0.000000e+00 0.000000e+00 1.000000e+00]
bn240401332 UNCERT [1.2989594e-03 9.9705398e-01 7.4252393e-04 9.0454391e-04]
bn240413660 UNCERT [8.9359622e-07 9.9832207e-01 4.3546748e-07 1.6765746e-03]
bn220625879 UNCERT [0. 1. 0. 0.]
bn230217562 UNCERT [2.1724562e-10 9.9999988e-01 4.1925343e-08 1.2099414e-07]
bn170815526 UNCERT [3.9071844e-09 8.7091173e-17 2.0060738e-13 1.0000000e+00]
bn121015799 UNCERT [0.00563264 0.9584837  0.01211066 0.02377299]
bn190804362 UNCERT [9.9524215e-05 9.9900419e-01 1.1792999e-04 7.7835389e-04]
bn230525821 UNCERT [0.00789735 0.930737   0.00874191 0.05262378]
bn240401460 UNCERT [1.76228932e-05 9.99867678e-01 1.31132565e-05 1.01500897e-04]
bn240419585 UNCERT [1.8522146e-04 9.9916923e-01 4.3016864e-04 2.1538667e-04]
bn151106440 UNCERT [0.00335664 0.99037397 0.00125738 0.00501203]
bn220418167 UNRELOC [0. 0. 0. 1.]
bn151107579 UNCERT [0.00598793 0.92594886 0.04199491 0.02606826]
bn240507981 UNCERT [0.00437958 0.9892686  0.0019323  0.00441954]
bn130831726 UNRELOC [3.2295824e-13 1.0233137e-12 1.5180862e-14 1.0000000e+00]
bn230921573 UNCERT [0.00132429 0.99529785 0.0015622  0.00181567]
bn210525596 UNCERT [6.7336368e-04 9.9812692e-01 4.4533925e-04 7.5441966e-04]
bn210223581 UNCERT [9.4225239e-03 9.7284532e-01 2.7647138e-05 1.7704474e-02]
bn121210642 UNCERT [2.6104017e-04 9.5683667e-05 9.9964261e-01 6.3888280e-07]
bn240516754 UNCERT [4.3375439e-05 9.9970621e-01 1.5025196e-04 1.0008260e-04]
bn220609520 UNCERT [0.0010186  0.9659889  0.02431135 0.00868118]
bn150317798 UNCERT [1.1433680e-07 9.9260992e-01 1.1400433e-06 7.3887394e-03]
bn220307298 UNCERT [0.00391264 0.9570165  0.0308274  0.0082435 ]
bn141227445 UNCERT [0.00188685 0.9920168  0.00172465 0.00437177]
bn231006376 UNCERT [0.00121128 0.9893638  0.00660429 0.00282062]
bn231226939 UNCERT [0.00240973 0.90839136 0.07228293 0.01691604]
bn130502009 UNCERT [0.01667328 0.9653098  0.00344104 0.0145758 ]
bn100303194 UNCERT [3.0411263e-03 9.9685413e-01 2.8494289e-05 7.6266166e-05]
bn170813463 UNCERT [4.7803496e-04 9.9766302e-01 3.0464123e-04 1.5542570e-03]
bn171115576 UNCERT [2.5058498e-24 4.8543498e-24 8.1622541e-25 1.0000000e+00]
bn240102760 UNCERT [0.0000000e+00 5.3969844e-38 0.0000000e+00 1.0000000e+00]
bn190619235 UNCERT [7.2660757e-04 9.9118912e-01 5.6856968e-03 2.3985826e-03]
bn210831148 UNCERT [1.1976730e-25 1.0000000e+00 4.3776417e-28 3.6851385e-26]
bn221207040 UNCERT [8.5554458e-23 1.0000000e+00 1.1048221e-25 2.4310808e-23]
bn170712283 UNCERT [8.2470091e-05 9.9959415e-01 2.4965124e-05 2.9835053e-04]
bn171017234 UNRELOC [0.0000000e+00 0.0000000e+00 3.4845587e-36 1.0000000e+00]
bn220214693 UNCERT [0.0085454  0.98115313 0.00284786 0.00745368]
bn141210586 UNRELOC [2.3124339e-05 9.9935323e-01 3.4398020e-06 6.2020042e-04]
bn171104946 UNRELOC [4.5661884e-11 8.0338129e-13 3.9588469e-13 1.0000000e+00]
bn211221163 UNCERT [5.2330202e-20 1.0000000e+00 1.4252354e-20 1.8197219e-20]
bn230509259 UNCERT [0. 0. 0. 1.]
bn230912087 UNCERT [8.1477148e-05 9.9979013e-01 6.9431720e-05 5.8985472e-05]
bn230714211 UNCERT [1.11144116e-04 9.98962998e-01 4.42407763e-05 8.81650660e-04]
bn240318304 UNCERT [0.0024645  0.9877415  0.00211852 0.00767546]
bn200116786 UNCERT [5.7141820e-04 9.9929965e-01 8.2831255e-05 4.6100820e-05]
bn220723610 UNCERT [1.07070164e-07 9.99933600e-01 2.10170086e-07 6.60961086e-05]
bn240324730 UNCERT [0.000000e+00 0.000000e+00 4.049717e-08 1.000000e+00]
bn081223486 UNCERT [9.8568287e-33 5.5189809e-33 3.6952548e-36 1.0000000e+00]
bn211104209 UNRELOC [2.266711e-28 0.000000e+00 0.000000e+00 1.000000e+00]
bn240608992 UNCERT [2.7162695e-16 1.0000000e+00 4.9937090e-19 2.4284292e-16]
bn130816205 BELOWHZ [1.3922764e-05 3.9196684e-06 1.6938735e-06 9.9998045e-01]
bn150318079 UNCERT [2.6771826e-07 1.2255597e-07 1.2088822e-09 9.9999964e-01]
bn110403668 UNCERT [1.6622143e-06 9.9999213e-01 1.9144709e-06 4.3058963e-06]
bn240326553 UNCERT [1.4772436e-24 1.0000000e+00 8.2509107e-25 3.4077036e-25]
bn130421569 UNRELOC [2.4655971e-25 4.9841757e-26 2.3482204e-28 1.0000000e+00]
bn220603728 UNCERT [6.4165861e-04 9.9567837e-01 2.7283952e-03 9.5160078e-04]
bn120727408 UNCERT [4.9204082e-05 9.9800569e-01 1.1094350e-04 1.8342264e-03]
'''

# Process each line of the data
for line in data.strip().split('\n'):
    match = re.match(pattern, line)
    if match:
        event_names.append(f"bn{match.group(1)}")
        event_types.append(match.group(2))
        pred_values = np.array([float(x) for x in match.group(3).split()])
        # Round off to 4 decimal places using np.around
        pred_values = np.around(pred_values, decimals=4)
        
        predictions.append(pred_values)

# Create the DataFrame
df = pd.DataFrame({
    'event_name': event_names,
    'event_type': event_types,
    'pred': predictions
})

# Display the first few rows of the DataFrame
print(df.head())

# Optional: Save the DataFrame to a CSV file
# df.to_csv('event_data.csv', index=False)

    event_name event_type                              pred
0  bn240323372     UNCERT  [0.9924, 0.0005, 0.0014, 0.0057]
1  bn110102432     UNCERT        [0.9998, 0.0, 0.0, 0.0002]
2  bn240628601     UNCERT     [0.92, 0.073, 0.0004, 0.0066]
3  bn221018317     UNCERT  [0.9062, 0.0797, 0.0012, 0.0129]
4  bn120716030    UNRELOC  [0.9888, 0.0007, 0.0004, 0.0102]


In [3]:
# Add 'max_index' column
df['max_index'] = df['pred'].apply(lambda x: np.argmax(x))

print(df.head())

    event_name event_type                              pred  max_index
0  bn240323372     UNCERT  [0.9924, 0.0005, 0.0014, 0.0057]          0
1  bn110102432     UNCERT        [0.9998, 0.0, 0.0, 0.0002]          0
2  bn240628601     UNCERT     [0.92, 0.073, 0.0004, 0.0066]          0
3  bn221018317     UNCERT  [0.9062, 0.0797, 0.0012, 0.0129]          0
4  bn120716030    UNRELOC  [0.9888, 0.0007, 0.0004, 0.0102]          0


In [6]:
def display_rows_by_max_index(df, index):
    """
    Display all rows from the DataFrame where max_index equals the given index.
    
    Parameters:
    df (pandas.DataFrame): The DataFrame to search
    index (int): The max_index value to match
    """
    matching_rows = df[df['max_index'] == index]
    if matching_rows.empty:
        print(f"No rows found with max_index = {index}")
    else:
        print(f"Rows with max_index = {index}:")
        # Set precision to 3 for all floats
        np.set_printoptions(precision=3)
        print(matching_rows)
        print(f"\nTotal rows: {len(matching_rows)}")

filtered_df = df[df['event_type'] == 'UNCERT']
filtered_df = filtered_df[filtered_df['max_index'] != 1]
# filtered_df = filtered_df[filtered_df['max_index'] != 2]
# Display rows for each possible max_index
for i in range(4):  # Assuming pred has 4 elements
    print("\n" + "="*50)
    display_rows_by_max_index(filtered_df, i)


Rows with max_index = 0:
     event_name event_type                              pred  max_index
0   bn240323372     UNCERT  [0.9924, 0.0005, 0.0014, 0.0057]          0
1   bn110102432     UNCERT        [0.9998, 0.0, 0.0, 0.0002]          0
2   bn240628601     UNCERT     [0.92, 0.073, 0.0004, 0.0066]          0
3   bn221018317     UNCERT  [0.9062, 0.0797, 0.0012, 0.0129]          0
6   bn211014070     UNCERT  [0.9982, 0.0003, 0.0005, 0.0011]          0
7   bn091022752     UNCERT              [1.0, 0.0, 0.0, 0.0]          0
8   bn230913474     UNCERT              [1.0, 0.0, 0.0, 0.0]          0
9   bn220115787     UNCERT     [0.9996, 0.0001, 0.0, 0.0003]          0
10  bn240524337     UNCERT  [0.9914, 0.0057, 0.0007, 0.0023]          0
11  bn240618962     UNCERT     [0.9702, 0.0001, 0.0, 0.0297]          0
12  bn110130345     UNCERT        [0.9999, 0.0, 0.0, 0.0001]          0
13  bn091208623     UNCERT        [0.9999, 0.0001, 0.0, 0.0]          0
14  bn160929461     UNCERT        [0.9

In [8]:
# Ensure the base directory exists
base_directory = r'/home/arpan-john/Desktop/cans'
type_dict = {0:'GRBC', 1:'TGFC', 2 : 'SGRC', 3 : 'SFLAREC'}

os.makedirs(base_directory, exist_ok=True)

for _, row in filtered_df.iterrows():

    event = row['event_name']
    year = '20' + event[2:4] + "/"

    cantype = type_dict[row['max_index']]
    if cantype!='SGRC':
        continue
    # Create folder name
    folder_name = f"{cantype}_{row['event_name']}"
    folder_path = os.path.join(base_directory, folder_name)
    
    # Create folder
    os.makedirs(folder_path, exist_ok=True)

    # URL of the tte file to download
    url = 'wget -q -nH --no-check-certificate --cut-dirs=7 -r -l0 -c -N -np -R "index*" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/' + year + event + '/current/'

    # Construct the wget command
    print('downloading: ', folder_path)
    tools.run_wget_download(url, folder_path)
    
    
    # Check for .rsp files and delete folder if none exist
    if not any(file.endswith('.rsp') for file in os.listdir(os.path.join(folder_path,'current'))):
        shutil.rmtree(folder_path)
        print(f"Deleted empty folder: {folder_path}")
    else:
        print(f"Folder retained: {folder_path}")

downloading:  /home/arpan-john/Desktop/cans/SGRC_bn230218466
download start
Downloaded Desktop" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2023/bn230218466/current/ -P /home/arpan-john/Desktop/cans/SGRC_bn230218466 to /home/arpan-john/Desktop/cans/SGRC_bn230218466
Deleted empty folder: /home/arpan-john/Desktop/cans/SGRC_bn230218466
downloading:  /home/arpan-john/Desktop/cans/SGRC_bn210818917
download start
Downloaded Desktop" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2021/bn210818917/current/ -P /home/arpan-john/Desktop/cans/SGRC_bn210818917 to /home/arpan-john/Desktop/cans/SGRC_bn210818917
Deleted empty folder: /home/arpan-john/Desktop/cans/SGRC_bn210818917
downloading:  /home/arpan-john/Desktop/cans/SGRC_bn121210642
download start
Downloaded Desktop" -erobots=off --retr-symlinks https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/2012/bn121210642/current/ -P /home/arpan-john/Desktop/cans/SG